In [0]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import udf, col, lower, regexp_replace
import re
import pandas as pd
import numpy as np
import string




In [0]:
df = spark.read.csv('/FileStore/tables/cyberbullydata.csv',inferSchema=True,header=True)
spark.conf.set("spark.sql.legacy.timeParserPolicy","Legacy")

df=df.select(['Insult','Comment'])

df.dropna()


Out[47]: DataFrame[Insult: int, Comment: string]

In [0]:
#text_length
#word_count
#unique_word_count
#mean_word_length
#punctuation_count

import pyspark.sql.functions as f
import string
from pyspark.sql.functions import array_contains
	 
df = df.withColumn("text_length", f.length("Comment"))
df = df.withColumn('word_count', f.size(f.split(f.col('Comment'), ' ')))
df = df.withColumn('uniqWords',f.size(f.array_distinct(f.split(f.col('Comment'), ' ')))) 
df = df.withColumn('mean_word_length',f.col('text_length')/f.col('word_count')) 

import string
#df.rdd.Comment.map(lambda x:len([c for c in str(x) if c in string.punctuation]))

#insult,comment, text_length,word_count,uniqWords,mean_word_length
punc_rdd=df.rdd.map(lambda x:(x[0],x[1],x[2],x[3],x[4],x[5],len([c for c in str(x) if c in string.punctuation])))
#df['punctuation_count']=
df = punc_rdd.toDF(["insult","comment","text_length","word_count","uniqWords","mean_word_length","countpunc"])
df.printSchema()
df.show()

root
-- insult: long (nullable = true)
-- comment: string (nullable = true)
-- text_length: long (nullable = true)
-- word_count: long (nullable = true)
-- uniqWords: long (nullable = true)
-- mean_word_length: double (nullable = true)
-- countpunc: long (nullable = true)

+------+--------------------+-----------+----------+---------+-----------------+---------+
insult| comment|text_length|word_count|uniqWords| mean_word_length|countpunc|
+------+--------------------+-----------+----------+---------+-----------------+---------+
 0|"""THE DRUDGE REP...| 189| 27| 26| 7.0| 75|
 0|"""@ian21\xa0""Ro...| 47| 6| 6|7.833333333333333| 28|
 1|"""Agree with Ala...| 69| 14| 12|4.928571428571429| 24|
 0|"""Really?\\n\\nI...| 70| 10| 10| 7.0| 32|
 0|"""Really suck is...| 29| 5| 5| 5.8| 25|
 0|"""...after a cou...| 56| 9| 8|6.222222222222222| 33|
 0|"""Matthews mento...| 52| 8| 8| 6.5| 30|
 0|"""How about we a...| 81| 15| 15| 5.4| 27|
 1|"""To engage in a...| 177| 31| 28|5.709677419354839| 33|
 0|"""Same reason no...| 45| 8| 8| 5.625| 23|
 0| """Well| 7| 1| 1| 7.0| 23|
 0|"""In reality GWB...| 97| 16| 16| 6.0625| 28|
 0|"""because the zi...| 68| 13| 13|5.230769230769231| 29|
 0|"""Thank you. Un...| 84| 16| 15| 5.25| 30|
 0|"""CNN supports H...| 108| 11| 11|9.818181818181818| 43|
 0|"""What is your p...| 29| 5| 5| 5.8| 26|
 0|"""Name a corpora...| 49| 8| 8| 6.125| 27|
 0| """You know| 11| 2| 2| 5.5| 23|
 0|"""then they shou...| 51| 9| 8|5.666666666666667| 27|
 0|"""Richards passe...| 59| 7| 7|8.428571428571429| 32|
+------+--------------------+-----------+----------+---------+-----------------+---------+
only showing top 20 rows

In [0]:
 
import re
import string
from pyspark.sql.functions import udf, col, lower, regexp_replace


def preProcess(iter):
    
        
        # remove extra space
        regex_ws=re.compile("\s+")
        ret=regex_ws.sub(" ",iter)
        ret=ret.replace("&amp;","&").replace("&lt;","<").replace("&gt;",">").replace("\\x","")
        #text="".join([word for word in text if word not in string.punctuation])
        
        #Replace URL
        regexp="(https?:\/\/(?:www\.|(?!www)|(?:xmlns\.))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
        ret=re.sub(regexp,"url",ret)
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        ret= url_pattern.sub(r'', ret)
  
        #replace @addresses
        regexp='@[A-z0-9_]+'
        ret=re.sub(regexp,"@refertoperson",ret)

        #Bring everything to lowercase
        ret=ret.lower()
 
        #removing punctuations 
        ret="".join([char for char in ret if char not in string.punctuation])

        return ret
    
preProcudf=udf(preProcess)
df = df.withColumn('Comment', preProcudf('Comment'))
df.show(5)
#display(df.selectExpr("Insult", "Comment"))
        

+------+--------------------+-----------+----------+---------+-----------------+---------+
insult| Comment|text_length|word_count|uniqWords| mean_word_length|countpunc|
+------+--------------------+-----------+----------+---------+-----------------+---------+
 0|the drudge report...| 189| 27| 26| 7.0| 75|
 0|refertopersonroge...| 47| 6| 6|7.833333333333333| 28|
 1|agree with alan y...| 69| 14| 12|4.928571428571429| 24|
 0|reallynni see mar...| 70| 10| 10| 7.0| 32|
 0|really suck isnt ...| 29| 5| 5| 5.8| 25|
+------+--------------------+-----------+----------+---------+-----------------+---------+
only showing top 5 rows

In [0]:
from sparknlp.annotator import DocumentNormalizer, Tokenizer, LemmatizerModel, StopWordsCleaner, Normalizer
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline

# Creates document formats
documentAssembler = (DocumentAssembler()\
                     .setInputCol('Comment')\
                     .setOutputCol('Document')
                    )
doc_norm_patterns = ["<[^>]>","_","https?:\/\/.*[\r\n]*", '"', r"\\n", r"\\u\d+","0_0"]

# Cleans Documents
documentNormalizer = DocumentNormalizer() \
    .setInputCols("Document") \
    .setOutputCol("NormalizedDocument") \
    .setAction("clean") \
    .setPatterns(doc_norm_patterns) \
    .setReplacement(" ") \
    .setPolicy("pretty_all") \
    .setLowercase(True)

# Tokenizes
tokenizer = Tokenizer()\
    .setInputCols(["NormalizedDocument"])\
    .setOutputCol("Token").fit(df)
# Normalizer - cleans tokens remaining
normalizer = Normalizer() \
    .setInputCols(["Token"]) \
    .setOutputCol("Normalized") \
    .setCleanupPatterns([r"\\x.\d",r"\\","@[^\s]+"])

# # Spell Checker
# spellChecker = ContextSpellCheckerModel \
#     .pretrained() \
#     .setTradeoff(12.0) \
#     .setInputCols("Token") \
#     .setOutputCol("Checked")


# Lemmatize
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(["Normalized"]) \
    .setOutputCol("Lemma")

# Remove Stop Words
stopWords = StopWordsCleaner() \
     .setInputCols(["Lemma"]) \
     .setOutputCol("CleanTokens") \
     .setCaseSensitive(False)

pipeline = Pipeline().setStages([
    documentAssembler,
    documentNormalizer,
    tokenizer,
    normalizer,
    lemmatizer,
    stopWords
])

#sample_data = spark.createDataFrame(df)
# data = spark.createDataFrame([[text]]).toDF("Text")
pipelineModel = pipeline.fit(df)
result = pipelineModel.transform(df)
display(result.selectExpr("Insult", "Comment","CleanTokens.result","text_length","word_count","uniqWords","mean_word_length","countpunc"))
df=result.selectExpr("Insult", "Comment","CleanTokens.result","text_length","word_count","uniqWords","mean_word_length","countpunc")


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][OK!]

Insult,Comment,result,text_length,word_count,uniqWords,mean_word_length,countpunc
0,the drudge reportnnnnyou wont see this story on foxfag forum because they suck biacks and gay 0bama all the way to the cracknnnnnnnnon tuesday rep darrell issa,"List(drudge, reportnnnnyou, wont, see, story, foxfag, forum, suck, biacks, gay, 0bama, way, cracknnnnnnnnon, tuesday, rep, darrell, issa)",189,27,26,7.0,75
0,refertopersonroger clemens is the fucking man,"List(refertopersonroger, clemens, fucking, man)",47,6,6,7.833333333333333,28
1,agree with alan you are an extremest idiot you are not american,"List(agree, alan, extreme, idiot, american)",69,14,12,4.928571428571429,24
0,reallynni see marc lamont hill on various programs quite often,"List(reallynni, see, marc, lamont, hill, various, program, quite, often)",70,10,10,7.0,32
0,really suck isnt the word,"List(really, suck, isnt, word)",29,5,5,5.8,25
0,after a couple guys like him rape him first,"List(couple, guy, like, rape, first)",56,9,8,6.222222222222222,33
0,matthews mentored perry at usc bodes well,"List(matthew, mentore, perry, usc, bode, well)",52,8,8,6.5,30
0,how about we accept facts and patiently wait for you to back up your claim,"List(accept, fact, patiently, wait, back, claim)",81,15,15,5.4,27
1,to engage in an intelligent debate with you is like debating to a retarded person its useless it looks like youre bent on disregarding the efforts of the government,"List(engage, intelligent, debate, like, debate, retarded, person, useless, look, like, youre, bend, disregard, effort, government)",177,31,28,5.709677419354839,33
0,same reason no one cared obama did cocaine,"List(reason, one, care, obama, cocaine)",45,8,8,5.625,23


In [0]:
#https://spark.apache.org/docs/3.2.0/ml-features.html#tf-idf\


###### method 2 #######
#hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
#featurizedData = hashingTF.transform(clean_text)

# Term Frequency Vectorization  - Option 2 (CountVectorizer)    : 
cv = CountVectorizer(inputCol="words", outputCol="rawFeatures", vocabSize = 1000)
cvmodel = cv.fit(clean_text)
featurizedData = cvmodel.transform(clean_text)

vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData) # TFIDF

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2926630070338551> in <module> 
 7 
 8 # Term Frequency Vectorization - Option 2 (CountVectorizer) : 
 ----> 9 cv = CountVectorizer ( inputCol = "words" , outputCol = "rawFeatures" , vocabSize = 1000 ) 
 10 cvmodel = cv . fit ( clean_text ) 
 11 featurizedData = cvmodel . transform ( clean_text ) 

 NameError : name 'CountVectorizer' is not defined

In [0]:
from pyspark.ml.feature import NGram

# Define NGram transformer
ngram = NGram(n=2, inputCol="unigrams", outputCol="bigrams")

# Create bigram_df as a transform of unigram_df using NGram tranformer
production_df = ngram.transform(production_df)

# Display
production_df.printSchema()
production_df.show()

In [0]:
rdd=spark.sparkContext.parallelize(data)rdd=spark.sparkContext.parallelize(data)from pyspark.sql.functions import col, size

production_df = production_df.where(size(col("bigrams")) >= 2)

# Display
production_df.printSchema()
production_df.show()

In [0]:


train_data,test_data=df.randomSplit([0.7,0.3])

In [0]:

pip install nltk
# Import stemmer library
import nltk
from nltk.stem.porter import *

# Instantiate stemmer object
stemmer = PorterStemmer(inputCol='MeaningfulWords',
                       outputCol='OrgWords')



print(stemmer.stem(t))

In [0]:
#tokenize
tokenizer=Tokenizer(inputCol='Comment',outputCol='Commentwords')
tok_train=tokenizer.transform(df)
display(tok_train.selectExpr("Insult", "Comment","Commentwords"))
